## Import exiobase3 in brightway2
by Massimo Pizzol, Jan19

In [1]:
import pandas as pd
import numpy as np
from pyxlsb import open_workbook as open_xlsb
from brightway2 import *

In [2]:
projects.set_current('test_exio3_import')
#bw2setup() # need to import biosphere3

In [71]:
databases

Databases dictionary with 1 object(s):
	biosphere3

In [5]:
#this takes some minutes, but seems the only way to import xlsb files in python
HIO_Arr = []
with open_xlsb("exiobase_3.3.15_hsut_2011/Exiobase_MR_HIOT_2011_v3_3_15_by_prod_tech.xlsb") as wb:
        # Read the sheet to array first and convert to pandas first for quick access
        with wb.get_sheet(2) as sheet:
            for row in sheet.rows(sparse = True):
                #print(row[0])
                vals = [item.v for item in row]
                HIO_Arr.append(vals)

HIO = pd.DataFrame(HIO_Arr)

In [6]:
# create some unique activity IDs ('country-code1-code2', e.g. 'AU-c01.i-P_CATL')
exio_IDs = HIO.iloc[4:, 0].values + "-" + HIO.iloc[4:, 2].values + "-" + HIO.iloc[4:, 3].values 
n_act = len(exio_IDs)

# other metadata
exio_countries = HIO.iloc[4:, 0].values # lists
exio_names = HIO.iloc[4:, 1].values
exio_codes1 = HIO.iloc[4:, 2].values 
exio_codes2 = HIO.iloc[4:, 3].values 
exio_units = HIO.iloc[4:, 4].values

# just the simplest IO table with only column names and row names, used just below for filtering of zero values
exio_data = HIO.iloc[4:, 5:] # pandas dataframe
exio_data.columns = exio_IDs
exio_data.index = exio_IDs
exio_data.head()

,AU-c01.a-P_PARI,AU-c01.b-P_WHEA,AU-c01.c-P_OCER,AU-c01.d-P_FVEG,AU-c01.e-P_OILS,AU-c01.f-P_SUGB,AU-c01.g-P_FIBR,AU-c01.h-P_OTCR,AU-c01.i-P_CATL,AU-c01.j-P_PIGS,...,WM-c90.5.b-P_LANP,WM-c90.5.c-P_LANL,WM-c90.5.d-P_LANI,WM-c90.5.e-P_LANT,WM-c90.5.f-P_LANW,WM-c91-P_ORGA,WM-c92-P_RECR,WM-c93-P_OSER,WM-c95-P_PRHH,WM-c99-P_EXTO
AU-c01.a-P_PARI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AU-c01.b-P_WHEA,0,0,0,0,0,0,0,0,678333,155894,...,0,0,0,0,0,0,0,0,0,0
AU-c01.c-P_OCER,0,0,109422,0,0,0,0,0,2.27671e+06,823210,...,0,0,0,0,0,0,0,0,0,0
AU-c01.d-P_FVEG,0,0,0,77617,0,0,0,0,1546.66,254.436,...,0,0,0,0,0,0,0,0,0,0
AU-c01.e-P_OILS,0,0,0,0,0,0,-955759,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [247]:
#import pickle

In [248]:
#exio_data.to_pickle("./exio_data.pkl") 
# see here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html

In [255]:
#exio_data2 = pd.read_pickle("./exio_data.pkl") # to open again

In [7]:
# filter out the zero values in the database
d = {c : exio_data.loc[exio_data[c] !=0 , c] for c in exio_IDs } # creates a dict with name, df with values != 0

In [165]:
db_name = 'exio3HIO'
if db_name in databases:
    del databases[db_name]  #just to make sure there isnt another one

In [166]:
# Create the database as a dict (didn't take long).
data = [{
        'name': exio_names[i],
        'code': exio_IDs[i],
        'key': (db_name, exio_IDs[i]),
        'exiobase_code1': exio_codes1[i],
        'exiobase_code2': exio_codes2[i],
        'database': db_name,
        'location': exio_countries[i],
        'unit': exio_units[i],
        'type': 'process',
        'exchanges': [{
                'input': (db_name, d[exio_IDs[i]].index[j]),
                'amount': d[exio_IDs[i]][j],
                'type': 'technosphere'
            }
        for j in range(d[exio_IDs[i]].shape[0])]
    } for i in range(n_act)]

exio_d = {obj['key']: obj for obj in data}

In [167]:
# This takes time! 15 min on my mac
# write the entire IO table
exio3 = Database(db_name)
exio3.write(exio_d)

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:25:33


Title: Writing activities to SQLite3 database:
  Started: 03/29/2019 12:48:59
  Finished: 03/29/2019 13:14:33
  Total time elapsed: 00:25:33
  CPU %: 84.70
  Memory %: 21.37


In [ ]:
# I shuold add a line here to pickle this object as it takes a lot to recreate it.

In [169]:
# check if it's alright
myact = exio3.get('AU-c01.i-P_CATL')
for exc in list(myact.exchanges())[0:4]:
        print(exc)
# seems alright

Exchange: 678332.8470145611 tonnes 'Cultivation of wheat' (tonnes, AU, None) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 2276711.5604753764 tonnes 'Cultivation of cereal grains nec' (tonnes, AU, None) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 1546.661506585483 tonnes 'Cultivation of vegetables, fruit, nuts' (tonnes, AU, None) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 19355.104385711475 tonnes 'Cattle farming' (tonnes, AU, None) to 'Cattle farming' (tonnes, AU, None)>


In [ ]:
pickle_out = open("dict.pickle","wb")
pickle.dump(example_dict, pickle_out)
pickle_out.close()

In [170]:
# Import principal production vector from xlsb file (fast)
PPV_Arr = []
with open_xlsb("exiobase_3.3.15_hsut_2011/Exiobase_MR_HIOT_2011_v3_3_15_by_prod_tech.xlsb") as wb:
        # Read the sheet to array first and convert to pandas first for quick access
        with wb.get_sheet(3) as sheet:
            for row in sheet.rows(sparse = True):
                #print(row[0])
                vals = [item.v for item in row]
                PPV_Arr.append(vals)
                
PPV = pd.DataFrame(PPV_Arr)
p_prod = PPV.iloc[5,1:] # just the numeric values
p_prod.index = exio_IDs

In [171]:
# set the principal production as reference flow for each activity (type = production)
for act in exio3: 
    exc = act.new_exchange(input = (db_name, act['code']), amount = p_prod[act['code']], unit = act['unit'], type = 'production')
    exc.save()

In [172]:
# check if it's alright
myact = exio3.get('AU-c01.i-P_CATL')
for exc in list(myact.exchanges()):
    if exc['type'] == 'production':
        print(exc)
# seems alright

Exchange: 1145042.8284939448 tonnes 'Cattle farming' (tonnes, AU, None) to 'Cattle farming' (tonnes, AU, None)>


In [173]:
# import data from the extensions file, but just the emissions for this test
ArrExt_emiss = []
with open_xlsb(r'exiobase_3.3.15_hsut_2011/MR_HSUT_2011_v3_3_15_extensions.xlsb') as wb:
        # Read the sheet to array first and convert to pandas first for quick access
        with wb.get_sheet('Emiss_act') as sheet:
            for row in sheet.rows(sparse = True):
                #print(row[0])
                vals = [item.v for item in row]
                ArrExt_emiss.append(vals)
                
Ext_emiss = pd.DataFrame(ArrExt_emiss)
Ext_emiss.head(6)

,0,1,2,3,4,5,6,7,8,9,...,7865,7866,7867,7868,7869,7870,7871,7872,7873,7874
0,None,None,None,AU,AU,AU,AU,AU,AU,AU,...,WM,WM,WM,WM,WM,WM,WM,WM,WM,WM
1,None,None,None,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities...",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
2,None,None,None,i01.a,i01.b,i01.c,i01.d,i01.e,i01.f,i01.g,...,i90.5.b,i90.5.c,i90.5.d,i90.5.e,i90.5.f,i91,i92,i93,i95,i99
3,None,None,None,A_PARI,A_WHEA,A_OCER,A_FVEG,A_OILS,A_SUGB,A_FIBR,...,A_LANP,A_LANL,A_LANI,A_LANT,A_LANW,A_ORGA,A_RECR,A_OSER,A_PRHH,A_EXTO
4,"Carbon dioxide, fossil",tonnes,air,28333.4,1.64288e+06,399460,188343,72481.9,70129.7,129804,...,107.655,857.972,417.751,2976.82,18.7294,7.73808e+06,6.56843e+06,1.72631e+06,6.51238e+06,0
5,N2O,tonnes,air,44.0359,9494.75,3888.77,8401.16,465.883,1159.71,1166.56,...,0,0,0,0,0,0,0,0,0,0


In [174]:
#Import mapping file to match Exiobase flows to BW2 ones.
df_flow_map = pd.read_csv("list_biosphere_flows_BW2.csv")
def find_flow(row):
    if row!=None:
        try:
            return df_flow_map.loc[df_flow_map["Exiobase name"]==row, "code"].values[0]
        except:
            print("No match for {}.".format(row))     
Ext_emiss.iloc[:, 0] = Ext_emiss.iloc[:, 0].apply(find_flow)

No match for Chlordane.
No match for Chlordecone.
No match for Dieldrin.
No match for Endrin.
No match for Heptachlor.
No match for Hexabr.-biph..
No match for Mirex.
No match for Toxaphene.
No match for HCH.
No match for DDT.
No match for PCB.
No match for PBDEs.
No match for Benzo-[k]-fluoranthene.
No match for HCB.
No match for PCDD/F (dioxins and furans).
No match for TSP (total suspended particulate).
No match for C.
No match for other emissions.
No match for Other emissions nec.


In [40]:
# I have only manually found these three ones for now, will have to do the same work for the remaining ones
#emission_codes = {'Carbon dioxide, fossil': 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46',
#                  'N2O':'afd6d670-bbb0-4625-9730-04088a5b035e',
#                  'CH4':'70ef743b-3ed5-4a6d-b192-fb6d62378555'}

In [175]:
# Again some formatting necessary for the next step
Ext_emiss_s0 = Ext_emiss[~Ext_emiss.iloc[:, 0].isnull()]
Ext_emiss_s = Ext_emiss_s0.iloc[:,3:]
Ext_emiss_s.columns = [exio_IDs]
Ext_emiss_s.index = list(Ext_emiss_s0.iloc[:,0].values)
Ext_emiss_s.head()

,AU-c01.a-P_PARI,AU-c01.b-P_WHEA,AU-c01.c-P_OCER,AU-c01.d-P_FVEG,AU-c01.e-P_OILS,AU-c01.f-P_SUGB,AU-c01.g-P_FIBR,AU-c01.h-P_OTCR,AU-c01.i-P_CATL,AU-c01.j-P_PIGS,...,WM-c90.5.b-P_LANP,WM-c90.5.c-P_LANL,WM-c90.5.d-P_LANI,WM-c90.5.e-P_LANT,WM-c90.5.f-P_LANW,WM-c91-P_ORGA,WM-c92-P_RECR,WM-c93-P_OSER,WM-c95-P_PRHH,WM-c99-P_EXTO
349b29d1-3e58-4c66-98b9-9d1a076efd2e,28333.4,1.64288e+06,399460,188343,72481.9,70129.7,129804,40717.9,2.85021e+06,5906.03,...,107.655,857.972,417.751,2976.82,18.7294,7.73808e+06,6.56843e+06,1.72631e+06,6.51238e+06,0
20185046-64bb-4c09-a8e7-e8a9e144ca98,44.0359,9494.75,3888.77,8401.16,465.883,1159.71,1166.56,3.02647,76262,400.721,...,0,0,0,0,0,0,0,0,0,0
0795345f-c7ae-410c-ad25-1845784c75f5,26526.8,151.077,121.413,56.9706,18.6846,6.71137,12.3297,6.79134,1.64257e+06,3470.59,...,7637.85,240.614,0.0156301,1017.53,216.28,0,0,0,0,0
e87f039d-d9ce-4242-ac18-c9284cc80f3d,45.6862,3803.28,2997.73,2262.58,563.046,10201.5,84.4774,87450.3,1039.99,152.9,...,0,0,0,0,0,0,0,0,0,0
e2ccc98a-6d4e-443b-b11f-d16f32782833,0.411631,34.2674,27.0095,20.3857,5.07302,91.9148,0.761138,787.924,9.37024,1.37763,...,0,0,0,0,0,0,0,0,0,0


In [178]:
# Add biosphere3 flows to the database
for act in exio3:
    mydf = Ext_emiss_s[act['code']][Ext_emiss_s[act['code']] != 0].dropna()
    for i in range(mydf.shape[0]):
        #print(i)
        exc = act.new_exchange(input = ('biosphere3', mydf.index[i]), amount = float(mydf.iloc[i].values)*1000, type = 'biosphere')
        exc.save()

In [179]:
# check if it's alright
myact = exio3.get('AU-c01.i-P_CATL')
for exc in list(myact.exchanges()):
    if exc['type'] == 'biosphere':
        print(exc)
# Seems alright

Exchange: 2850213222.8075724 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 76262009.76250708 kilogram 'Dinitrogen monoxide' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 1642572105.724754 kilogram 'Methane, fossil' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 1039986.18940033 kilogram 'Ethane, 1,1,1,2-tetrafluoro-, HFC-134a' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 9370.23543377383 kilogram 'Perfluoropentane' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 3.4525487535397206 kilogram 'Sulfur hexafluoride' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 175823752.32185462 kilogram 'Nitric oxide' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes, AU, None)>
Exchange: 223479.57051041836 kilogram 'Sulfur oxides' (kilogram, None, ('air',)) to 'Cattle farming' (tonnes,

In [217]:
# now try some calculations
functional_unit = {exio3.get('AU-c01.i-P_CATL'): 1}
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
lca = LCA(functional_unit, mymethod)
lca.lci(factorize = True) # To allow for reusing results? Not sure
lca.lcia()

In [235]:
print(lca.score)

-inf


In [245]:
lca.redo_lcia(functional_unit) # faster once matrix is inverted

In [251]:
inventory = pd.DataFrame(lca.inventory.toarray())

In [300]:
sum(inventory.iloc[4,:]) # 4.7 tons of CO2 emissions for 1 ton of cattle farming in Australia

4778.508129092075

In [ ]:
# End

In [ ]:
# all the stuff below is to figure out why ee get -inf after LCIA.

In [252]:
inventory

,0,1,2,3,4,5,6,7,8,9,...,7862,7863,7864,7865,7866,7867,7868,7869,7870,7871
0,1.788003e-03,1.462253e+00,4.531764e+00,4.681382e-01,1.942375e-01,1.120148e-02,1.355044e-04,7.209571e-05,5.172375e+02,2.123796e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,8.462516e-10,3.684741e-07,6.518365e-07,1.443800e-08,3.630620e-08,1.488785e-09,2.239411e-11,2.587176e-09,9.909743e-07,2.059936e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
2,1.311680e-08,5.710832e-06,9.936449e-06,2.193713e-07,5.582247e-07,1.467897e-08,3.486554e-10,5.856830e-10,1.532761e-05,1.385443e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
3,2.272872e-04,9.895692e-02,1.721777e-01,3.801242e-03,9.672857e-03,2.543566e-04,6.041485e-06,1.014869e-05,5.683881e+03,2.433864e-05,...,1.060873e-04,1.477571e-10,5.410711e-11,2.713402e-11,3.014496e-06,5.068778e-06,3.100089e-07,2.940101e-07,7.652255e-08,0.0
4,9.675945e-02,4.426597e+01,8.140668e+01,1.815542e+00,5.399763e+00,1.178770e-01,2.613307e-03,1.301422e-01,2.532155e+03,1.023658e-02,...,5.169710e-07,6.185233e-06,4.091851e-07,7.694187e-06,8.066188e-08,2.834465e-03,7.481647e-03,1.493670e-03,1.592517e-03,0.0
5,2.380400e-04,1.038913e-01,1.549456e-01,3.438747e-03,8.481318e-03,3.905098e-04,6.309359e-06,2.052142e-03,2.835845e-01,1.954776e-05,...,2.182413e-09,1.032109e-10,0.000000e+00,1.564915e-10,5.542270e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
6,6.998193e-09,3.047685e-06,4.231200e-06,9.344301e-08,2.369135e-07,8.154716e-09,1.859596e-10,6.574127e-09,8.194106e-06,7.609454e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
7,1.503843e-04,2.558275e-01,7.924998e-01,8.098332e-02,3.470740e-02,1.949283e-03,2.348599e-05,9.673172e-06,6.775185e+01,6.945461e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
8,1.560200e-04,1.024758e-01,6.109135e-01,2.181018e-02,4.194580e-02,1.714706e-02,1.700759e-06,2.795077e-01,9.239330e-01,2.650136e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
9,9.058996e-02,4.070619e-03,2.474304e-02,5.491702e-04,1.391964e-03,1.128075e-05,2.482312e-07,2.170640e-05,1.459276e+03,6.015369e-03,...,3.667787e-05,1.734617e-06,1.530960e-11,2.630016e-06,9.314565e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


In [234]:
sum(inventory.iloc[4,:]) # 4.7 tons of CO2 emissions for 1 ton of cattle farming in Australia

4778.508129092075

In [37]:
# end

In [239]:
lca.technosphere_matrix # OK, 7872 activities and same amount of products

<7872x7872 sparse matrix of type '<class 'numpy.float64'>'
	with 14473829 stored elements in Compressed Sparse Row format>

In [240]:
lca.biosphere_matrix # OK, 29 bioshphere exchanges

<29x7872 sparse matrix of type '<class 'numpy.float64'>'
	with 159794 stored elements in Compressed Sparse Row format>

In [241]:
lca.inventory # OK, 29 biosphere exchanges

<29x7872 sparse matrix of type '<class 'numpy.float64'>'
	with 159410 stored elements in Compressed Sparse Row format>

In [242]:
exio3.metadata

{'backend': 'sqlite',
 'depends': ['biosphere3'],
 'modified': '2019-03-29T13:36:59.364247',
 'number': 7872,
 'processed': '2019-03-29T13:48:19.011703',
 'searchable': True}

In [243]:
projects.output_dir

'/Users/massimo/Library/Application Support/Brightway3/test_exio3_import.4da78af77b9063a17cca6d84758289e3/output'

In [246]:
backup_project_directory('test_exio3_import')

Creating project backup archive - this could take a few minutes...


In [299]:
inventory

,0,1,2,3,4,5,6,7,8,9,...,7862,7863,7864,7865,7866,7867,7868,7869,7870,7871
0,1.788003e-03,1.462253e+00,4.531764e+00,4.681382e-01,1.942375e-01,1.120148e-02,1.355044e-04,7.209571e-05,5.172375e+02,2.123796e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,8.462516e-10,3.684741e-07,6.518365e-07,1.443800e-08,3.630620e-08,1.488785e-09,2.239411e-11,2.587176e-09,9.909743e-07,2.059936e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
2,1.311680e-08,5.710832e-06,9.936449e-06,2.193713e-07,5.582247e-07,1.467897e-08,3.486554e-10,5.856830e-10,1.532761e-05,1.385443e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
3,2.272872e-04,9.895692e-02,1.721777e-01,3.801242e-03,9.672857e-03,2.543566e-04,6.041485e-06,1.014869e-05,5.683881e+03,2.433864e-05,...,1.060873e-04,1.477571e-10,5.410711e-11,2.713402e-11,3.014496e-06,5.068778e-06,3.100089e-07,2.940101e-07,7.652255e-08,0.0
4,9.675945e-02,4.426597e+01,8.140668e+01,1.815542e+00,5.399763e+00,1.178770e-01,2.613307e-03,1.301422e-01,2.532155e+03,1.023658e-02,...,5.169710e-07,6.185233e-06,4.091851e-07,7.694187e-06,8.066188e-08,2.834465e-03,7.481647e-03,1.493670e-03,1.592517e-03,0.0
5,2.380400e-04,1.038913e-01,1.549456e-01,3.438747e-03,8.481318e-03,3.905098e-04,6.309359e-06,2.052142e-03,2.835845e-01,1.954776e-05,...,2.182413e-09,1.032109e-10,0.000000e+00,1.564915e-10,5.542270e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
6,6.998193e-09,3.047685e-06,4.231200e-06,9.344301e-08,2.369135e-07,8.154716e-09,1.859596e-10,6.574127e-09,8.194106e-06,7.609454e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
7,1.503843e-04,2.558275e-01,7.924998e-01,8.098332e-02,3.470740e-02,1.949283e-03,2.348599e-05,9.673172e-06,6.775185e+01,6.945461e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
8,1.560200e-04,1.024758e-01,6.109135e-01,2.181018e-02,4.194580e-02,1.714706e-02,1.700759e-06,2.795077e-01,9.239330e-01,2.650136e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
9,9.058996e-02,4.070619e-03,2.474304e-02,5.491702e-04,1.391964e-03,1.128075e-05,2.482312e-07,2.170640e-05,1.459276e+03,6.015369e-03,...,3.667787e-05,1.734617e-06,1.530960e-11,2.630016e-06,9.314565e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


In [258]:
for i in range(29):
    print(sum(inventory.iloc[i,:]))

541.3891146212192
0.00027290449913813683
0.00038061822351426145
9540.034090586281
4778.508129092075
4.769566988926675
0.0012488094587046377
71.46503769118873
-8.973624558068575e+306
1470.9149892809241
34.10081776839044
0.0031417281261341477
0.7522036336763963
0.6072331068988154
5.573652543314428e-05
-4.311305202287872e+301
0.00010197640278693861
0.0006455328411661683
0.0030204654554968923
8.872186753329812e-05
0.0009897655538221447
654.1674322544351
0.002607112238106106
56.082393242289164
-2.4770602549863526e+305
6.91170426435169
0.0003076253838604141
0.00012639371302127674
184.98300081750946


In [259]:
inventory

,0,1,2,3,4,5,6,7,8,9,...,7862,7863,7864,7865,7866,7867,7868,7869,7870,7871
0,1.788003e-03,1.462253e+00,4.531764e+00,4.681382e-01,1.942375e-01,1.120148e-02,1.355044e-04,7.209571e-05,5.172375e+02,2.123796e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
1,8.462516e-10,3.684741e-07,6.518365e-07,1.443800e-08,3.630620e-08,1.488785e-09,2.239411e-11,2.587176e-09,9.909743e-07,2.059936e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
2,1.311680e-08,5.710832e-06,9.936449e-06,2.193713e-07,5.582247e-07,1.467897e-08,3.486554e-10,5.856830e-10,1.532761e-05,1.385443e-09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
3,2.272872e-04,9.895692e-02,1.721777e-01,3.801242e-03,9.672857e-03,2.543566e-04,6.041485e-06,1.014869e-05,5.683881e+03,2.433864e-05,...,1.060873e-04,1.477571e-10,5.410711e-11,2.713402e-11,3.014496e-06,5.068778e-06,3.100089e-07,2.940101e-07,7.652255e-08,0.0
4,9.675945e-02,4.426597e+01,8.140668e+01,1.815542e+00,5.399763e+00,1.178770e-01,2.613307e-03,1.301422e-01,2.532155e+03,1.023658e-02,...,5.169710e-07,6.185233e-06,4.091851e-07,7.694187e-06,8.066188e-08,2.834465e-03,7.481647e-03,1.493670e-03,1.592517e-03,0.0
5,2.380400e-04,1.038913e-01,1.549456e-01,3.438747e-03,8.481318e-03,3.905098e-04,6.309359e-06,2.052142e-03,2.835845e-01,1.954776e-05,...,2.182413e-09,1.032109e-10,0.000000e+00,1.564915e-10,5.542270e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
6,6.998193e-09,3.047685e-06,4.231200e-06,9.344301e-08,2.369135e-07,8.154716e-09,1.859596e-10,6.574127e-09,8.194106e-06,7.609454e-10,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
7,1.503843e-04,2.558275e-01,7.924998e-01,8.098332e-02,3.470740e-02,1.949283e-03,2.348599e-05,9.673172e-06,6.775185e+01,6.945461e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
8,1.560200e-04,1.024758e-01,6.109135e-01,2.181018e-02,4.194580e-02,1.714706e-02,1.700759e-06,2.795077e-01,9.239330e-01,2.650136e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
9,9.058996e-02,4.070619e-03,2.474304e-02,5.491702e-04,1.391964e-03,1.128075e-05,2.482312e-07,2.170640e-05,1.459276e+03,6.015369e-03,...,3.667787e-05,1.734617e-06,1.530960e-11,2.630016e-06,9.314565e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0


In [267]:
Ext_emiss_s

,AU-c01.a-P_PARI,AU-c01.b-P_WHEA,AU-c01.c-P_OCER,AU-c01.d-P_FVEG,AU-c01.e-P_OILS,AU-c01.f-P_SUGB,AU-c01.g-P_FIBR,AU-c01.h-P_OTCR,AU-c01.i-P_CATL,AU-c01.j-P_PIGS,...,WM-c90.5.b-P_LANP,WM-c90.5.c-P_LANL,WM-c90.5.d-P_LANI,WM-c90.5.e-P_LANT,WM-c90.5.f-P_LANW,WM-c91-P_ORGA,WM-c92-P_RECR,WM-c93-P_OSER,WM-c95-P_PRHH,WM-c99-P_EXTO
349b29d1-3e58-4c66-98b9-9d1a076efd2e,28333.4,1.64288e+06,399460,188343,72481.9,70129.7,129804,40717.9,2.85021e+06,5906.03,...,107.655,857.972,417.751,2976.82,18.7294,7.73808e+06,6.56843e+06,1.72631e+06,6.51238e+06,0
20185046-64bb-4c09-a8e7-e8a9e144ca98,44.0359,9494.75,3888.77,8401.16,465.883,1159.71,1166.56,3.02647,76262,400.721,...,0,0,0,0,0,0,0,0,0,0
0795345f-c7ae-410c-ad25-1845784c75f5,26526.8,151.077,121.413,56.9706,18.6846,6.71137,12.3297,6.79134,1.64257e+06,3470.59,...,7637.85,240.614,0.0156301,1017.53,216.28,0,0,0,0,0
e87f039d-d9ce-4242-ac18-c9284cc80f3d,45.6862,3803.28,2997.73,2262.58,563.046,10201.5,84.4774,87450.3,1039.99,152.9,...,0,0,0,0,0,0,0,0,0,0
e2ccc98a-6d4e-443b-b11f-d16f32782833,0.411631,34.2674,27.0095,20.3857,5.07302,91.9148,0.761138,787.924,9.37024,1.37763,...,0,0,0,0,0,0,0,0,0,0
35d1dff5-b535-4628-9826-4a8fce08a1f2,0.000151669,0.0126261,0.00995188,0.00751131,0.0018692,0.0338669,0.000280448,0.290318,0.00345255,0.000507599,...,0,0,0,0,0,0,0,0,0,0
17f7e776-7934-54af-8c3c-ed0247cb697b,161.631,10181.4,2635.85,2275.56,384.829,484.204,894.36,49.0336,175824,3705.68,...,3.38928,1.78861,4.29307,0.763323,1.14545,0,0,0,0,0
ba5fc0b6-770b-4da1-9b3f-e3b5087f07cd,49.7487,2745.29,631.567,294.795,97.0551,113.281,224.305,3.81292,223.48,10.3757,...,0.0388287,0.0204908,0.0491827,0.00874486,0.0131226,0,0,0,0,0
87883a4e-1e3e-4c9d-90c0-f1bea36f8014,523.568,54269.9,22237.2,48564.4,2607.28,6664.21,6730.56,22.5568,582207,12253.3,...,0,0,0,0,0,0,0,0,0,0
d3260d0e-8203-4cbb-a45a-6a13131a5108,44.9458,2473.58,4884.89,8749.31,7285.02,40.9481,20525.4,233.708,22370.7,1324.36,...,1977.88,1615.28,2091.99,25.01,828.171,2242.87,2870.35,4545.95,95.6896,0


In [298]:
Ext_emiss_s.loc['87883a4e-1e3e-4c9d-90c0-f1bea36f8014']

AU-c01.a-P_PARI           523.568
AU-c01.b-P_WHEA           54269.9
AU-c01.c-P_OCER           22237.2
AU-c01.d-P_FVEG           48564.4
AU-c01.e-P_OILS           2607.28
AU-c01.f-P_SUGB           6664.21
AU-c01.g-P_FIBR           6730.56
AU-c01.h-P_OTCR           22.5568
AU-c01.i-P_CATL            582207
AU-c01.j-P_PIGS           12253.3
AU-c01.k-P_PLTR           39581.6
AU-c01.l-P_OMEA           8361.14
AU-c01.m-P_OANP         0.0005458
AU-c01.n-P_MILK           69749.5
AU-c01.o-P_WOOL           57082.9
AU-c01.w.1-P_MANC         64626.5
AU-c01.w.2-P_MANB               0
AU-c02-P_FORE             8.87462
AU-c05-P_FISH             1.51362
AU-c10-P_COAL             9.25084
AU-c11.a-P_COIL           3.86044
AU-c11.b-P_GASE           16.4497
AU-c11.c-P_OGPL                 0
AU-c12-P_ORAN            0.281076
AU-c13.1-P_IRON           25.4892
AU-c13.20.11-P_COPO       6.69303
AU-c13.20.12-P_NIKO       1.11224
AU-c13.20.13-P_ALUO       10.1929
AU-c13.20.14-P_PREO       4.04208
AU-c13.20.15-P

In [277]:
ciccio = Ext_emiss_s['AU-c01.a-P_PARI']

In [287]:
ciccio[ciccio != 0].dropna().index[8]

'87883a4e-1e3e-4c9d-90c0-f1bea36f8014'

In [292]:
Database('biosphere3').get(ciccio[ciccio != 0].dropna().index[8])

'Ammonia' (kilogram, None, ('air',))

In [293]:
Database('biosphere3').get(ciccio[ciccio != 0].dropna().index[-5])

'Indeno(1,2,3-cd)pyrene' (kilogram, None, ('air',))